#INSTALLATION

In [1]:
! pip install transformers datasets sacrebleu rouge_score py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 

##CONNECT TO HUGGINGFACE ACCOUNT

In [2]:
from huggingface_hub import notebook_login

notebook_login()

#IMPORT LIBRARY

In [3]:
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
from transformers import AutoTokenizer, BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset, load_metric
import pandas as pd
import tqdm
import numpy as np

#LOAD DATASET

In [5]:
dataset_samsum = load_dataset('samsum', split = "train")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


Train dataset size has 14k7 rows

In [6]:
dataset_samsum

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

Split data to train and test data

In [7]:
dataset_samsum = dataset_samsum.train_test_split(test_size=0.2)

In [8]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 11785
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 2947
    })
})

In [9]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

#PRE-TRAINED MODEL BART-BASED

In [10]:
checkpoint = 'facebook/bart-base'

TOKENIZER AND BARD MODEL

In [11]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = BartForConditionalGeneration.from_pretrained(checkpoint)

In [12]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [13]:
tokenized_samsum = dataset_samsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/11785 [00:00<?, ? examples/s]

Map:   0%|          | 0/2947 [00:00<?, ? examples/s]

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [15]:
rouge_metric = load_metric("rouge")

<ipython-input-15-46a4432edece>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


#TRAINING MODEL

In [16]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.1 MB/s eta 0:00:00


In [17]:
import evaluate

rouge = evaluate.load("rouge")
def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [18]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

In [20]:
from transformers import TrainingArguments, Trainer


training_args = Seq2SeqTrainingArguments(

    output_dir='bart-samsum',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,

)

In [21]:
trainer = Seq2SeqTrainer(
                  model=model, 
                  args=training_args,
                  tokenizer=tokenizer, 
                  data_collator=data_collator,
                  train_dataset=tokenized_samsum["train"], 
                  eval_dataset=tokenized_samsum["test"],
                  compute_metrics=compute_metrics)

Cloning https://huggingface.co/ThanhDVi/bart-samsum into local empty directory.


In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.808200,1.561297,0.476300,0.241200,0.404300,0.404100,17.933200
2,1.560900,1.520557,0.482700,0.248500,0.408200,0.408500,18.316900
3,1.422800,1.500818,0.485100,0.255700,0.413800,0.413700,17.985100
4,1.313100,1.507078,0.483500,0.254600,0.412800,0.413100,17.981700


TrainOutput(global_step=11788, training_loss=1.5493487179137748, metrics={'train_runtime': 3673.5825, 'train_samples_per_second': 12.832, 'train_steps_per_second': 3.209, 'total_flos': 8051548495011840.0, 'train_loss': 1.5493487179137748, 'epoch': 4.0})

In [23]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/532M [00:00<?, ?B/s]

Upload file runs/May07_11-17-57_bf9ee392341b/events.out.tfevents.1683458307.bf9ee392341b.439.0:   0%|         …

To https://huggingface.co/ThanhDVi/bart-samsum
   3c2bd1f..75025c6  main -> main

   3c2bd1f..75025c6  main -> main

To https://huggingface.co/ThanhDVi/bart-samsum
   75025c6..1332e28  main -> main

   75025c6..1332e28  main -> main



'https://huggingface.co/ThanhDVi/bart-samsum/commit/75025c67f7254ed24ca8062566341ab52c909edc'

In [24]:
from google.colab import drive
drive.mount('/content/drive')
     

Mounted at /content/drive


In [25]:
## Save model
model.save_pretrained("/content/drive/MyDrive/Saved models/bart-samsum-model")

In [26]:
## Save tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Saved models/tokenizer") 

('/content/drive/MyDrive/Saved models/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Saved models/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Saved models/tokenizer/vocab.json',
 '/content/drive/MyDrive/Saved models/tokenizer/merges.txt',
 '/content/drive/MyDrive/Saved models/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/Saved models/tokenizer/tokenizer.json')

##TEST

In [27]:
dataset_samsum = load_dataset("samsum")     

  0%|          | 0/3 [00:00<?, ?it/s]

In [31]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Saved models/tokenizer")

In [32]:
sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

In [33]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

pipe = pipeline("summarization", model="/content/drive/MyDrive/Saved models/bart-samsum-model",tokenizer=tokenizer)

In [34]:

print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Hannah can't find Betty's number. Larry called her last time they were at the park together.
